In [34]:
# optimization of spacings between heliostats

In [35]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt

In [36]:
# https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html

import scipy
from scipy.optimize import minimize_scalar
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy import optimize

## Generation of layout

In [37]:
def findSpacings(xMin, xMax, xStep, isCentered, nMax):
    # possible positions
    x0 = 0. if isCentered else xStep/2;
    ansP = np.arange(x0, xMax, xStep)
    ansN = -np.arange(-(x0 - xStep), -xMin, xStep) 
    ans = np.concatenate((ansN, ansP))
    # select
    ans = sorted(ans, key=lambda x: abs(x))
    ans = ans[0:min(nMax, len(ans))]
    # sort
    ans = np.sort(ans)
    return ans

In [38]:
def writeRow(letter, spacings, y, z, f):
    ans = ""
    for (i, x) in enumerate(spacings):
        ans += "{name},{sx:.3f} {sy:.3f} {sz:.3f},{sf:.3f}\n".format(
        name=letter + str(i + 1).zfill(2),
        sx=x,
        sy=y,
        sz=z,
        sf=f
    )
    return ans

In [39]:
nFunc=0
def writeLayout(s):
    out = "Name,Position [m],Focus [m]\n"
    out += writeRow("A", findSpacings(-35.11, 27., s[0], True, 7), 46, 10, 46)
    out += writeRow("B", findSpacings(-42.5, 30.33, s[1], False, 8), 52, 14.5, 52)
    out += writeRow("C", findSpacings(-50.77, 15.83, s[2], True, 5), 58, 19.5, 58)
    #print(out)
    fileOut = open("heliostatsB.csv","w")
    fileOut.write(out)
    fileOut.close()
    global nFunc
    if not os.path.exists('temp'):
        os.makedirs('temp')
    fileOut = open("temp/heliostatsB{i}.csv".format(i=nFunc),"w")
    fileOut.write(out)
    fileOut.close()

In [40]:
findSpacings(-50.77, 15.83, 8.9, True, 5)

array([-26.7, -17.8,  -8.9,   0. ,   8.9])

## Ray tracing

In [41]:
bat = "cmd /k {tonatiuh} -i={file}".format(
    tonatiuh = os.getcwd() + "/../../../bin/Tonatiuh-Application.exe",
    file="makeField.tnhpps"
)

def tnEta(x):
    # input
    writeLayout(x)
    
    # simulation
    os.system(bat)
    
    # output
    f = open("out.csv", "r")
    txt = f.read()
    f.close()
    return float(txt) 

In [42]:
eta = tnEta([6.5, 6.5, 6.5])
eta

0.2861958913805672

## Scalar

In [43]:
def tnFunc(x):
    return -tnEta([x, x, x])

In [44]:
#res = minimize_scalar(tnFunc, method='bounded', bounds=(5., 10.), options={'disp':3,'maxiter':20,'xatol':0.01})

In [45]:
#res

## Multivariate

In [46]:
scipy.optimize.show_options(solver="minimize", method="cobyla")

Minimize a scalar function of one or more variables using the
Constrained Optimization BY Linear Approximation (COBYLA) algorithm.

Options
-------
rhobeg : float
    Reasonable initial changes to the variables.
tol : float
    Final accuracy in the optimization (not precisely guaranteed).
    This is a lower bound on the size of the trust region.
disp : bool
    Set to True to print convergence messages. If False,
    `verbosity` is ignored as set to 0.
maxiter : int
    Maximum number of function evaluations.
catol : float
    Tolerance (absolute) for constraint violations


In [47]:
def tnFunc(x):
    global nFunc
    nFunc = nFunc + 1
    ans = tnEta(x)
    print('iteration {0:d}: {1:.3f}, {2:.3f}, {3:.3f}, f={4:.4f}'.format(nFunc, x[0], x[1], x[2], ans))
    return -ans

In [48]:
nFunc = 0
res = minimize(
    tnFunc, np.array([6.5, 6.5, 6.5]), constraints=[], tol=0.01,
    method='cobyla', options={'maxiter': 1000, 'disp': True, 'catol': 0.1}
)

iteration 1: 6.500, 6.500, 6.500, f=0.2741
iteration 2: 7.500, 6.500, 6.500, f=0.2861
iteration 3: 7.500, 7.500, 6.500, f=0.2831
iteration 4: 7.500, 6.500, 7.500, f=0.2832
iteration 5: 8.445, 6.265, 6.273, f=0.2902
iteration 6: 9.014, 5.673, 5.702, f=0.2606
iteration 7: 8.588, 6.743, 6.254, f=0.2915
iteration 8: 8.644, 6.736, 6.497, f=0.2905
iteration 9: 8.866, 6.863, 5.856, f=0.2817
iteration 10: 8.124, 6.921, 6.311, f=0.2844
iteration 11: 8.702, 6.734, 6.476, f=0.2888
iteration 12: 8.656, 6.734, 6.150, f=0.2896
iteration 13: 8.595, 6.805, 6.253, f=0.2790
iteration 14: 8.558, 6.622, 6.257, f=0.2916
iteration 15: 8.497, 6.634, 6.259, f=0.2826
iteration 16: 8.559, 6.622, 6.288, f=0.2906
iteration 17: 8.617, 6.607, 6.241, f=0.2963
iteration 18: 8.671, 6.592, 6.213, f=0.2892
iteration 19: 8.609, 6.577, 6.242, f=0.2859
iteration 20: 8.644, 6.663, 6.234, f=0.2910
iteration 21: 8.627, 6.578, 6.233, f=0.2914
iteration 22: 8.614, 6.610, 6.226, f=0.2791
iteration 23: 8.627, 6.607, 6.271, f=0.28

In [49]:
res

     fun: -0.2938610148585623
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 27
  status: 1
 success: True
       x: array([8.62241995, 6.60903949, 6.24919826])

In [50]:
tnFunc(res.x)

iteration 28: 8.622, 6.609, 6.249, f=0.2836


-0.2835651712493677